In [0]:
schema_sales = "date STRING, region STRING, manager STRING, product STRING, amount INT"

In [0]:
data = spark.read.csv("/FileStore/tables/sales2.csv", schema = schema_sales, header = True, sep=";")

In [0]:
data.show()

+----------+------+-------+-------+------+
 date|region|manager|product|amount|
+----------+------+-------+-------+------+
01.01.2021| r1| m1| pr1| 400|
02.01.2021| r2| m2| pr2| 900|
03.01.2021| r3| m3| pr1| 500|
04.01.2021| r4| m1| pr2| 800|
05.01.2021| r5| m2| pr1| 200|
06.01.2021| r1| m3| pr2| 700|
07.01.2021| r2| m1| pr1| 100|
08.01.2021| r3| m2| pr2| 900|
09.01.2021| r4| m3| pr1| 100|
10.01.2021| r5| m1| pr2| 200|
11.01.2021| r1| m2| pr1| 500|
12.01.2021| r2| m3| pr2| 400|
13.01.2021| r3| m1| pr1| 400|
14.01.2021| r4| m2| pr2| 300|
15.01.2021| r5| m3| pr1| 200|
16.01.2021| r1| m1| pr2| 100|
17.01.2021| r2| m2| pr1| 300|
18.01.2021| r3| m3| pr2| 500|
19.01.2021| r4| m1| pr1| 500|
+----------+------+-------+-------+------+

In [0]:
data.printSchema()

root
-- date: string (nullable = true)
-- region: string (nullable = true)
-- manager: string (nullable = true)
-- product: string (nullable = true)
-- amount: integer (nullable = true)

In [0]:
from datetime import datetime
from pyspark.sql.functions import col, udf
from pyspark.sql.types import *
#Приводим столбец с датами к правильному формату
udf_date = udf(lambda x:datetime.strptime(x, "%d.%m.%Y"),DateType())
df = data.withColumn('date',udf_date(col('date')))

In [0]:
df.show(10)

+----------+------+-------+-------+------+
 date|region|manager|product|amount|
+----------+------+-------+-------+------+
2021-01-01| r1| m1| pr1| 400|
2021-01-02| r2| m2| pr2| 900|
2021-01-03| r3| m3| pr1| 500|
2021-01-04| r4| m1| pr2| 800|
2021-01-05| r5| m2| pr1| 200|
2021-01-06| r1| m3| pr2| 700|
2021-01-07| r2| m1| pr1| 100|
2021-01-08| r3| m2| pr2| 900|
2021-01-09| r4| m3| pr1| 100|
2021-01-10| r5| m1| pr2| 200|
+----------+------+-------+-------+------+
only showing top 10 rows

In [0]:
df.printSchema()

root
-- date: date (nullable = true)
-- region: string (nullable = true)
-- manager: string (nullable = true)
-- product: string (nullable = true)
-- amount: integer (nullable = true)

In [0]:
df.columns

Out[8]: ['date', 'region', 'manager', 'product', 'amount']

In [0]:
from pyspark.sql.functions import *
#Расчет количества уникальных менеджеров в датасете
df.select("manager").where(col("manager").isNotNull()).agg(countDistinct("manager").alias("DistintManager")).show()

+--------------+
DistintManager|
+--------------+
 3|
+--------------+

In [0]:
#Вывод имен менеджеров
df.select("manager").where(col("manager").isNotNull()).distinct().show()

+-------+
manager|
+-------+
 m3|
 m1|
 m2|
+-------+

In [0]:
#Вывод всех записи, которые относятся к менеджеру с кодом m3
df.select(_).where(col("manager")=="m3").show()

+----------+------+-------+-------+------+
 date|region|manager|product|amount|
+----------+------+-------+-------+------+
2021-01-03| r3| m3| pr1| 500|
2021-01-06| r1| m3| pr2| 700|
2021-01-09| r4| m3| pr1| 100|
2021-01-12| r2| m3| pr2| 400|
2021-01-15| r5| m3| pr1| 200|
2021-01-18| r3| m3| pr2| 500|
+----------+------+-------+-------+------+

In [0]:
#Расчет количества записей в датасете для каждого менеджера
df.select("manager").where(col("manager").isNotNull()).groupBy("manager").count().orderBy("count",ascending=False).show()

+-------+-----+
manager|count|
+-------+-----+
 m1| 7|
 m3| 6|
 m2| 6|
+-------+-----+

In [0]:
from pyspark.sql.functions import round
#Добавление нового расчетного столбца
df = df.withColumn('percentage',round(df.amount/100*2.5,1))

In [0]:
df.show(5)

+----------+------+-------+-------+------+----------+
 date|region|manager|product|amount|percentage|
+----------+------+-------+-------+------+----------+
2021-01-01| r1| m1| pr1| 400| 10.0|
2021-01-02| r2| m2| pr2| 900| 22.5|
2021-01-03| r3| m3| pr1| 500| 12.5|
2021-01-04| r4| m1| pr2| 800| 20.0|
2021-01-05| r5| m2| pr1| 200| 5.0|
+----------+------+-------+-------+------+----------+
only showing top 5 rows

In [0]:
#Фильтрация датасета
dfFilter = df.filter(df.percentage <=10)

In [0]:
dfFilter.show()

+----------+------+-------+-------+------+----------+
 date|region|manager|product|amount|percentage|
+----------+------+-------+-------+------+----------+
2021-01-01| r1| m1| pr1| 400| 10.0|
2021-01-05| r5| m2| pr1| 200| 5.0|
2021-01-07| r2| m1| pr1| 100| 2.5|
2021-01-09| r4| m3| pr1| 100| 2.5|
2021-01-10| r5| m1| pr2| 200| 5.0|
2021-01-12| r2| m3| pr2| 400| 10.0|
2021-01-13| r3| m1| pr1| 400| 10.0|
2021-01-14| r4| m2| pr2| 300| 7.5|
2021-01-15| r5| m3| pr1| 200| 5.0|
2021-01-16| r1| m1| pr2| 100| 2.5|
2021-01-17| r2| m2| pr1| 300| 7.5|
+----------+------+-------+-------+------+----------+

In [0]:
#Сборная статистика для числовых полей
df.select('amount').summary().show()

+-------+------------------+
summary| amount|
+-------+------------------+
 count| 19|
 mean|421.05263157894734|
 stddev|257.29133372377066|
 min| 100|
 25%| 200|
 50%| 400|
 75%| 500|
 max| 900|
+-------+------------------+

In [0]:
#Сортировка
df.sortWithinPartitions('region','manager','product',ascending=[True,True,True]).show()

+----------+------+-------+-------+------+----------+
 date|region|manager|product|amount|percentage|
+----------+------+-------+-------+------+----------+
2021-01-01| r1| m1| pr1| 400| 10.0|
2021-01-16| r1| m1| pr2| 100| 2.5|
2021-01-11| r1| m2| pr1| 500| 12.5|
2021-01-06| r1| m3| pr2| 700| 17.5|
2021-01-07| r2| m1| pr1| 100| 2.5|
2021-01-17| r2| m2| pr1| 300| 7.5|
2021-01-02| r2| m2| pr2| 900| 22.5|
2021-01-12| r2| m3| pr2| 400| 10.0|
2021-01-13| r3| m1| pr1| 400| 10.0|
2021-01-08| r3| m2| pr2| 900| 22.5|
2021-01-03| r3| m3| pr1| 500| 12.5|
2021-01-18| r3| m3| pr2| 500| 12.5|
2021-01-19| r4| m1| pr1| 500| 12.5|
2021-01-04| r4| m1| pr2| 800| 20.0|
2021-01-14| r4| m2| pr2| 300| 7.5|
2021-01-09| r4| m3| pr1| 100| 2.5|
2021-01-10| r5| m1| pr2| 200| 5.0|
2021-01-05| r5| m2| pr1| 200| 5.0|
2021-01-15| r5| m3| pr1| 200| 5.0|
+----------+------+-------+-------+------+----------+

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number
#Применение оконной функции
win = Window.partitionBy(df['region']).orderBy(df['amount'].desc())
df_rank = df.withColumn('rank',row_number().over(win).alias('rank'))
df_rank.show()

+----------+------+-------+-------+------+----------+----+
 date|region|manager|product|amount|percentage|rank|
+----------+------+-------+-------+------+----------+----+
2021-01-06| r1| m3| pr2| 700| 17.5| 1|
2021-01-11| r1| m2| pr1| 500| 12.5| 2|
2021-01-01| r1| m1| pr1| 400| 10.0| 3|
2021-01-16| r1| m1| pr2| 100| 2.5| 4|
2021-01-02| r2| m2| pr2| 900| 22.5| 1|
2021-01-12| r2| m3| pr2| 400| 10.0| 2|
2021-01-17| r2| m2| pr1| 300| 7.5| 3|
2021-01-07| r2| m1| pr1| 100| 2.5| 4|
2021-01-08| r3| m2| pr2| 900| 22.5| 1|
2021-01-03| r3| m3| pr1| 500| 12.5| 2|
2021-01-18| r3| m3| pr2| 500| 12.5| 3|
2021-01-13| r3| m1| pr1| 400| 10.0| 4|
2021-01-04| r4| m1| pr2| 800| 20.0| 1|
2021-01-19| r4| m1| pr1| 500| 12.5| 2|
2021-01-14| r4| m2| pr2| 300| 7.5| 3|
2021-01-09| r4| m3| pr1| 100| 2.5| 4|
2021-01-05| r5| m2| pr1| 200| 5.0| 1|
2021-01-10| r5| m1| pr2| 200| 5.0| 2|
2021-01-15| r5| m3| pr1| 200| 5.0| 3|
+----------+------+-------+-------+------+----------+----+

In [0]:
#Создание нового датасета
directory_product = spark.createDataFrame([('pr1','product1'),('pr2','product2')],['id','name'])

In [0]:
#Объединение датасетов
df_new = df.join(directory_product,df.product==directory_product.id,how='left')

In [0]:
df_new.show(5)

+----------+------+-------+-------+------+----------+---+--------+
 date|region|manager|product|amount|percentage| id| name|
+----------+------+-------+-------+------+----------+---+--------+
2021-01-01| r1| m1| pr1| 400| 10.0|pr1|product1|
2021-01-03| r3| m3| pr1| 500| 12.5|pr1|product1|
2021-01-05| r5| m2| pr1| 200| 5.0|pr1|product1|
2021-01-07| r2| m1| pr1| 100| 2.5|pr1|product1|
2021-01-09| r4| m3| pr1| 100| 2.5|pr1|product1|
+----------+------+-------+-------+------+----------+---+--------+
only showing top 5 rows

In [0]:
#Отбор столбцов и их переименование
df_final = df_new['date','region','manager','name','amount','percentage']
df_final = df_final.withColumnRenamed('name','product')

In [0]:
df_final.show(5)

+----------+------+-------+--------+------+----------+
 date|region|manager| product|amount|percentage|
+----------+------+-------+--------+------+----------+
2021-01-01| r1| m1|product1| 400| 10.0|
2021-01-03| r3| m3|product1| 500| 12.5|
2021-01-05| r5| m2|product1| 200| 5.0|
2021-01-07| r2| m1|product1| 100| 2.5|
2021-01-09| r4| m3|product1| 100| 2.5|
+----------+------+-------+--------+------+----------+
only showing top 5 rows

In [0]:
from pyspark.sql import functions as F
#Добавление расчетного столбца (столбец с условием)
df_final = df_final.withColumn('premium', F.when((df_final.percentage>=0)&(df_final.percentage<5),5).when((df_final.percentage>=5)&(df_final.percentage<10),10).otherwise(15))

In [0]:
df_final.show()

+----------+------+-------+--------+------+----------+-------+
 date|region|manager| product|amount|percentage|premium|
+----------+------+-------+--------+------+----------+-------+
2021-01-01| r1| m1|product1| 400| 10.0| 15|
2021-01-03| r3| m3|product1| 500| 12.5| 15|
2021-01-05| r5| m2|product1| 200| 5.0| 10|
2021-01-07| r2| m1|product1| 100| 2.5| 5|
2021-01-09| r4| m3|product1| 100| 2.5| 5|
2021-01-11| r1| m2|product1| 500| 12.5| 15|
2021-01-13| r3| m1|product1| 400| 10.0| 15|
2021-01-15| r5| m3|product1| 200| 5.0| 10|
2021-01-17| r2| m2|product1| 300| 7.5| 10|
2021-01-19| r4| m1|product1| 500| 12.5| 15|
2021-01-02| r2| m2|product2| 900| 22.5| 15|
2021-01-04| r4| m1|product2| 800| 20.0| 15|
2021-01-06| r1| m3|product2| 700| 17.5| 15|
2021-01-08| r3| m2|product2| 900| 22.5| 15|
2021-01-10| r5| m1|product2| 200| 5.0| 10|
2021-01-12| r2| m3|product2| 400| 10.0| 15|
2021-01-14| r4| m2|product2| 300| 7.5| 10|
2021-01-16| r1| m1|product2| 100| 2.5| 5|
2021-01-18| r3| m3|product2| 500| 12.5| 15|
+----------+------+-------+--------+------+----------+-------+

In [0]:
df_final.write.format("delta").save("/mnt/delta/sales3")

In [0]:
%sql
DROP TABLE IF EXISTS sales3;

CREATE TABLE sales3 USING DELTA LOCATION '/mnt/delta/sales3/'

In [0]:
%sql
select *
from sales3
limit 10

date,region,manager,product,amount,percentage,premium
2021-01-01,r1,m1,product1,400,10.0,15
2021-01-03,r3,m3,product1,500,12.5,15
2021-01-05,r5,m2,product1,200,5.0,10
2021-01-07,r2,m1,product1,100,2.5,5
2021-01-09,r4,m3,product1,100,2.5,5
2021-01-11,r1,m2,product1,500,12.5,15
2021-01-13,r3,m1,product1,400,10.0,15
2021-01-15,r5,m3,product1,200,5.0,10
2021-01-17,r2,m2,product1,300,7.5,10
2021-01-19,r4,m1,product1,500,12.5,15
